<a href="https://colab.research.google.com/github/Ramakrishna247/stmuart/blob/master/mymnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:

mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)= mnist.load_data()


11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = tf.keras.utils.normalize(x_train,axis=1)
x_test =tf.keras.utils.normalize(x_test,axis=1)
IMG_SIZE =28
x_trainr=np.array(x_train).reshape(-1,IMG_SIZE, IMG_SIZE, 1) #we are increasing  dimension to 4d  for kernel operation
x_testr= np.array(x_test).reshape(-1,IMG_SIZE,IMG_SIZE, 1)
from tensorflow.keras.models import Sequential   
from tensorflow.keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
model = keras.Sequential() 
model.add(Conv2D(64,(3,3),input_shape = x_trainr.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
#model.add(Dense(32))
#model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])
model.fit(x_trainr,y_train,epochs=5, validation_split=0.3)


Epoch 1/5
1313/1313 [==============================] - 66s 49ms/step - loss: 0.3012 - accuracy: 0.9064 - val_loss: 0.1144 - val_accuracy: 0.9643
Epoch 2/5
1313/1313 [==============================] - 65s 50ms/step - loss: 0.0975 - accuracy: 0.9704 - val_loss: 0.1031 - val_accuracy: 0.9682
Epoch 3/5
1313/1313 [==============================] - 63s 48ms/step - loss: 0.0699 - accuracy: 0.9791 - val_loss: 0.0883 - val_accuracy: 0.9723
Epoch 4/5
1313/1313 [==============================] - 63s 48ms/step - loss: 0.0558 - accuracy: 0.9826 - val_loss: 0.0607 - val_accuracy: 0.9816
Epoch 5/5
1313/1313 [==============================] - 63s 48ms/step - loss: 0.0436 - accuracy: 0.9862 - val_loss: 0.0592 - val_accuracy: 0.9819


In [ ]:
model.save("my_mnist.h5")

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations= [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open('mymodel.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpb5qgco6a/assets


INFO:tensorflow:Assets written to: /tmp/tmpb5qgco6a/assets


In [ ]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str






In [ ]:
# Write TFLite model to a C source (or header) file
with open('my_model.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, 'my_model'))
 